# Non-tradable variables as Factors
Factors do not have to be tradable portfolios. If all factors are non-tradeable, like macroeconomic variables for example, then this factor model can be used in order to identify their risk premia.

We start by importing the class and reading the data. For this example, we will use US GDP growth and inflation. As test assets, we will use the Fama-French 25 portfolios.

In [1]:
from empfin import NonTradableFactors, us_gdp, us_cpi, ff25p
import pandas as pd

# Read data
gdp = us_gdp()
cpi = us_cpi()
assets = ff25p(sub_rf=False)  # Not excess returns

Some data manipulation. We compute the quarterly growth of GDP and the CPI, and turn the Fama-French monthly portfolio returns into quarterly returns.

In [2]:
factors = pd.concat([gdp, cpi], axis=1).resample("QE").last().pct_change(fill_method=None).dropna()
assets = (1 + assets/100).cumprod().resample("QE").last().pct_change(1).dropna()

Run the model. The estimatior is iterative and this may take a few seconds.

In [3]:
ntf = NonTradableFactors(assets, factors, max_iter=1000)

Diff = 9.77007319491463e-09:  37%|███▋      | 371/1000 [00:00<00:01, 512.44it/s]  


In [4]:
ntf.B_con

array([[-0.18210574, -1.35803267],
       [ 0.54743513, -1.79998993],
       [ 0.50491751, -1.78499857],
       [ 0.71287505, -2.04038237],
       [ 1.12219808, -2.25102862],
       [-0.1502676 , -1.5224572 ],
       [ 0.31894731, -1.81011181],
       [ 0.52971308, -1.85346411],
       [ 0.80500985, -1.86972088],
       [ 0.58120156, -2.08066674],
       [-0.17438399, -1.52911443],
       [ 0.25336121, -1.80288763],
       [ 0.48198187, -1.6866214 ],
       [ 0.37541761, -1.89513959],
       [ 0.6061906 , -1.98934911],
       [-0.1382458 , -1.64438939],
       [ 0.06921785, -1.6206016 ],
       [ 0.50158101, -1.67428511],
       [ 0.35017099, -1.89289902],
       [ 0.82060316, -1.85106009],
       [ 0.1770478 , -1.58528341],
       [ 0.06912661, -1.40630926],
       [ 0.3225693 , -1.48809978],
       [ 0.61088519, -1.30574838],
       [ 0.44979668, -1.66575211]])

In [5]:
ntf.lambdas

GDP    0.001972
CPI   -0.018995
Name: Lambdas, dtype: float64

What these last values tells us is that the GDP factor has a risk premia of 0.2% per quarters, and inflation has -1.9% premium.

In [6]:
J, pval = ntf.lr_test()
print(f"The test statistic is stat is {round(J, 4)} with a p-value of {pval}")

The test statistic is stat is 18.0025 with a p-value of 0.7058421490580725


Not rejecting the null tells us that imposing the restrictions of the constrained model do not compromise its fit.